# CNN

# 전체 구조
완전 연결 (fully-connected): 인접하는 계층이 모든 뉴런과 결합되어 있는 형태 (Affine 계층)

층이 5개인 신경망:  
Input -> Affine | ReLU -> Affine | ReLU -> ... -> Affine | Softmax -> Output

CNN 구조:  
Input -> **Conv** | ReLU -> **Pooling** -> ... -> Affine | Softmax -> Output  
(출력에 가까운 층에서는 Affine | ReLU 구성 사용, 마지막 층은 Affine | Softmax 계층 사용)


# 합성곱 계층

## 완전연결 계층의 문제점
데이터의 형상이 무시됨 (3차원 이미지 데이터 입력 시 1차원으로 평탄화 필요)
 - MNIST 데이터 사용시 (1, 28, 28)인 데이터를 한 줄로 세워 입력하였음
 - 모든 입력 데이터를 동등한 뉴런으로 취급

합성곱 계층: 형상을 유지함 -> 이미지 데이터도 제대로 이해할 가능성이 있음

 - 특징 맵 (feature map): 합성곱 계층의 입출력 데이터
 - 입력 특징 맵 (input feature map): 합성곱 계층의 입력 데이터
 - 출력 특징 맵 (output feature map): 합성곱 계층의 출력 데이터

## 합성곱 연산
필터 연산에 해당

- 데이터와 필터의 형상을 (높이, 너비)로 표기
- 입력 데이터와 필터의 합성곱 연산을 통해 -> 출력 데이터를 만들어냄
- 필터의 윈도우를 이동해가며 입력 데이터에 적용  

- 가중치: 필터의 매개변수
- 편향: 1 X 1 크기를 필터의 모든 원소에 더함

## 패딩
입력 데이터 주변을 0으로 채우는 것  
패딩을 얼마나 크게 설정하느냐에 따라 입력 데이터의 크기가 달라짐  
주로 출력 크기를 조정할 목적으로 사용 (합성곱 연산 시 크기가 작아지지 않도록 패딩을 주고 공간 크기를 고정하여 다음 계층에 전달 가능)

## 스트라이드
필터를 적용하는 위치의 간격  
stride로 지정된 수 만큼 window가 이동

입력 크기 $(H, W)$, 필터 크기 $(FH, FW)$, 출력 크기 $(OH, OW), 패딩 $P$, 스트라이트 $S$ 인 경우:  
- $OH=\frac{H+2P-FH}{S} + 1$
- $OW=\frac{W+2P-FW}{S} + 1$

## 3차원 데이터의 합성곱 연산
입력 데이터별로 각각의 필터를 만들어서, 각각 합성곱을 수행한 후 합쳐서 최종 출력 데이터 형성

## 블록으로 생각하기
- 3차원 데이터의 합성곱 연산은 (C, H, W) * (C, FH, FW) = (OH, OW) 형식으로 생각할 수 있음
- 필터를 FN개 만들면, (FN, OH, OW) 형태로 여러 개릥 출력 데이터를 만들어낼 수 있음

필터의 가중치 데이터는 4차원 데이터로 (출력 채널 수, 입력 채널 수, 높이, 너비)로 구성됨

## 배치 처리
배치 처리를 지원하도록 (데이터 수, 채널 수, 높이, 너비) 순의 4차원 데이터로 저장

# 풀링 계층
세로 또는 가로 방향의 공간을 줄이는 연산
- 최대 풀링: 대상 영역의 크기 내에서 최댓값 원소를 꺼냄
- 평균 풀링: 대상 영역의 평균을 계산

일반적으로 최대 풀링을 사용

## 풀링 계층 특징
- 학습해야 할 매개변수 없음: 특별히 학습하지 않고 연산만 수행
- 채널 수 변동 없음: 채널마다 독립적으로 계산
- 입력 변화에 영향을 적게 받음: 입력 데이터가 조금 변해도 풀링의 결과가 변하지 않음


# 풀링 계층 구현

## 4차원 배열

In [1]:
import numpy as np

In [2]:
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [3]:
x[0].shape

(1, 28, 28)

In [4]:
x[1].shape

(1, 28, 28)

In [5]:
x[0][0]

array([[0.57033841, 0.4126144 , 0.46440991, 0.05090882, 0.146979  ,
        0.16917187, 0.85096052, 0.62514609, 0.8246459 , 0.93499759,
        0.7120633 , 0.52990339, 0.45776776, 0.74283701, 0.25749134,
        0.30482072, 0.53210152, 0.95409854, 0.67403629, 0.43693569,
        0.10498986, 0.1143595 , 0.3784315 , 0.09839352, 0.6634473 ,
        0.95103319, 0.1583857 , 0.23254135],
       [0.18352498, 0.60922419, 0.19718203, 0.2460608 , 0.83501189,
        0.13257388, 0.04130874, 0.26234181, 0.60107765, 0.49903405,
        0.57209155, 0.33117602, 0.50651308, 0.34002997, 0.64390879,
        0.38797431, 0.20344201, 0.73075606, 0.83542626, 0.55920245,
        0.26952166, 0.91339992, 0.82466575, 0.79335617, 0.80188007,
        0.88046712, 0.30254875, 0.40192833],
       [0.03092363, 0.89676332, 0.77232639, 0.60247059, 0.17192152,
        0.98495291, 0.43339294, 0.59165399, 0.86167288, 0.06185982,
        0.40872638, 0.12449557, 0.08426915, 0.21341341, 0.08715132,
        0.8503031 , 0.2560

## im2col로 데이터 전개
- numpy에서는 for문 사용시 성능이 떨어짐
- im2col을 이용하여 간단하게 구현

im2col을 3차원 입력 데이터에 적용하면 2차원 행렬로 변환  
이 때, stride에 따라 적용 영역이 겹치면 전개 후 원소 수가 많아져 메모리를 더 많이 소비함  
그러나 행렬 곱셈 연산 속도가 증가

## 합성곱 계층 구현

In [6]:
%cd ../

/Users/jinjae/Development/Deep-Learning-from-Scratch


In [7]:
from libs.utils import im2col

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(9, 75)


In [8]:
x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


In [10]:
# 합성곱 계층 구현
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2 * self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2 * self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        # reshape에 -1을 지정하면 나머지 원소 수는 변환 후에도 똑같이 유지되도록 묶어줌
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b

        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        return out

## 풀링 계층 구현
입력 데이터를 전개하여 최댓값을 구하고, reshape